In [16]:
#helper functions
import math
from sympy.ntheory import factorint
import numpy as np
from progressbar import *   
from tqdm import tqdm
import random

def gcd(x, y): 
    if y > x: return gcd(y, x)
    while(y): 
        x, y = y, x % y 
    return x

    
def erat_sieve(B):
    # Use Sieve of Eratosthenes to find all primes <= B
    is_prime = [True for i in range(B+1)]
    
    for i in range(math.ceil(math.sqrt(B))):
        if i == 0 or i == 1:
            is_prime[i] = False
        
        if is_prime[i]:
            for j in range(i*2, B + 1, i): #goes through all other multiples of i
                is_prime[j] = False

    
    return [p for p in range(B+1) if is_prime[p]]


def add_cols(M, a, b):
    
    return np.array([(M[i][a] + M[i][b]) % 2 for i in range(len(M))])

def find_pivot(M, col):
    for i, row in enumerate(range(len(M))):
        if M[row][col] == 1: return i
    
    return None


def reduce_matrix(M):
    #taken from https://www.sciencedirect.com/science/article/pii/074373159190115P
    #Assume M is K+1 x K
    
    ##Gaussian elimination of matrix
    K = len(M[0])
    
    marked = [0] * (K+1)
    pivots = [None] * K
    
    

    #for j in tqdm(range(K)):
    for j in range(K):
        i = find_pivot(M, j)
        if i is not None:
            marked[i] = True
            pivots[j] = i
            for k in range(K):
                if M[i][k] == 1 and j != k:
                    #print("k=", k)
                    M[:, k] = add_cols(M, j, k)
    
    #finding dependent rows
    rows = []
    #for i in tqdm(range(K+1)):
    for i in range(K+1):
        if not marked[i]:
            indices = [i]
            for j in range(K):
                if M[i][j] == 1:
                    indices.append(pivots[j])
            rows.append(indices)
                    
    return M, rows


def mod2(xs):
    return np.array([x % 2 for x in xs])


def find_a(a, p):
    """
    Find x such that x^2 = a (mod p)
    """
    t = 0
    while jacobi(t**2 - a, p) != -1:
        t = random.randint(0, p-1)
    
    p_2 = p**2
    
    i = (t**2 - a) % p_2
    j = (t + math.sqrt(i)) % p_2
    
    #x = j**(p+1)/2 % p**2
    """x = 1
    for i in range((p+1)//2):
        x  = (x * j) % p_2
    """
    
    x = bin_ladder(j, int((p+1)/2), p_2)
    
    return x 


def jacobi(a, m):
    """return jacobi(a/m)"""
    a = a % m 
    t = 1
    while a != 0:
        while a % 2 == 0:
            a /= 2
            if m % 8 in [3, 5]:
                t = -t
        
        a, m = m, a
        
        if a % 4 == 3 and m % 4 == 3:
            t = -t
        
        a = a % m
        
    if m == 1: return t
    
    return 0


def is_bsmooth(base, y):
    # compare to trial division
    #based off https://math.stackexchange.com/questions/182398/smooth-numbers-algorithm
    
    k = np.prod(base)
    
    
    g = gcd(y, k)
        
    while g > 1:
        #solve for y = rg^e
        r = y
        while r % g == 0:
            r /= g
        
        if r == 1:
            return True
        
        y = r
        
        g = gcd(y, k)
            
        
    
    return False
    

def b_factor(base, y):
    factors = np.zeros(len(base))
    
    while y > 1:
        for i, b in enumerate(base):
            if y % b == 0:
                factors[i] += 1
                
                y = y/b
                
    return factors
            
    
def bin_ladder(x, y, N):
    #compute x^y mod N
    
    y_string = bin(y)[2:]
    
    z = x
    
    for j in range(len(y_string)-1)[::-1]:

        z = z*z % N
        
        if y_string[j] == '1':
            z = (z*x) % N
    
    return z
    
    
    
    

                
                

In [24]:
def quadratic_sieve(n, B=None, verbose=False):
    

    if B is None: B = math.ceil(math.exp(math.sqrt(math.log(n) * math.log(math.log(n)))))
    
    if verbose: print("B =", B)
    

    
    smooths = []
    
    S = []
    
    print("finding primes...")
    b_primes = erat_sieve(B)
    K = len(b_primes)
    
    print("finding a_i's....")
    #A1 - find ± a_i where a_i^2 = n (mod p_i)
    a = [] 
    for p in tqdm(b_primes[1:]): #what to do for 2?
        a.append(find_a(n % p, p))
    
    
    
    
    ## Trying different x values
    
    print("sieving...")
    
    x = round(math.sqrt(n))
    
    widgets = ['b-smooth count: ', Percentage(), ' ', Bar(marker='-',left='[',right=']'),
           ' ', ETA(), ' ']

    pbar = ProgressBar(widgets=widgets, maxval=K+1)
    pbar.start()
    while len(smooths) <= K:
        y = (x**2) % n
        #print("here")
        if is_bsmooth(b_primes, y):
            factors = b_factor(b_primes, y)
            exps = np.zeros(len(b_primes))
            for i, f in enumerate(factors):
                exps[i] = factors[i] 
                
            if verbose: print("x =", x, ": ", exps)
            S.append((x, x**2 - n))
            smooths.append(exps)
        x += 1
        pbar.update(len(smooths)) #this adds a little symbol at each iteration
    pbar.finish()
    
    ## Gaussian elimination ....
    print("matrixing")
    M = np.array([mod2(s) for s in smooths])
    
    reduced_M, rows = reduce_matrix(M)
    
    #rows[0] contains the indices of M that sum to zero
    
    print("solving for x,y...")
    
    x = 1
    for r in rows[0]:
        x = (x * S[r][0]) % n
    
    y = 1 #how to use a_i's??
    for j in range(K):
        p = sum([smooths[r][j] for r in rows[0]])/2
        y = (y * b_primes[j] ** p) % n
    y = y % n
    
    d = gcd(x-y, n)
    
    print(n, "=", d, "*", n/d)
    assert n % d == 0
    
    return d
        
        
    
    
quadratic_sieve(46839566299936919234246726809, verbose=True)
#539873, B=19
#16921456439215439701
#46839566299936919234246726809

B = 16707785
finding primes...


  1%|          | 7865/1073664 [00:00<00:13, 78648.39it/s]

finding a_i's....


100%|██████████| 1073664/1073664 [00:19<00:00, 54368.96it/s]
b-smooth count: N/A% [                                        ] ETA:  --:--:-- 

sieving...


b-smooth count: N/A% [                                        ] ETA:  --:--:-- 

KeyboardInterrupt: 